<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/OpenAI_Whisper_ASR_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIのWhisper音声認識モデルをデモするWebアプリケーション

このColabノートブックは、[OpenAIのフリーWhisper音声認識モデル](https://openai.com/blog/whisper/)に音声ファイルを録音またはアップロードするためのものです。これは[@amrrsによるオリジナルのノートブック](https://github.com/amrrs/openai-whisper-webapp)をベースに、[Pete Warden](https://twitter.com/petewarden)によるドキュメントとテストファイルを追加したものです。

使用するには、Colabメニューから`Runtime->Run All`を選択してください。このノートブックをGitHubで見ている場合は、まず[このリンク](https://colab.research.google.com/github/petewarden/openai-whisper-webapp/blob/main/OpenAI_Whisper_ASR_Demo.ipynb)をたどってColabで開いてください。1分ほどすると、ページの下に`Record from microphone`リンクのボタンが表示されるはずです。これをクリックすると、マイクへのアクセス許可を求められるので、最大30秒間話してください。録音が終わったら、「録音停止」を押してください。録音ボタンの右側のボックスに、最初の30秒間のスピーチ原稿が表示されます。さらに書き起こしたい場合は、左のボックスの「クリア」をクリックし、最初からやり直してください。

このページの左側にあるフォルダアイコンを使って、自分の音声サンプルをアップロードすることもできます。ファイルをドラッグしてアップロードできるファイルシステムにアクセスできます。以下のいくつかのセルで、テープ起こしの実行例を見ることができます。

## Whisper のインストール

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

## モデルのロード

In [ ]:
import whisper

model = whisper.load_model("medium") # tiny, base, samll, medium, large, turbo


## GPUの確認

以下のように`device(type='cuda', index=0)`が出力されるはずです。もし表示されない場合、CPUのみのColabインスタンスを使用している可能性があり、動作が遅くなります。Runtime->Change Runtime Type`で修正してください。

In [ ]:
model.device

## テスト用音声ファイルのダウンロード

このリポジトリには、書き起こし機能を実行するためにあらかじめ録音された MP3 がいくつかあります。下に表示されているオーディオウィジェットで聴くことができます。

In [ ]:
!git clone https://github.com/petewarden/openai-whisper-webapp

In [ ]:
from IPython.display import Audio
Audio("/content/openai-whisper-webapp/mary.mp3")

In [ ]:
from IPython.display import Audio
Audio("/content/openai-whisper-webapp/daisy_HAL_9000.mp3")

## Transcribe 関数の定義

音声ファイルのパスを入力として受け取り、認識されたテキストを返します。

In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text


## 録音済みの音声でテストする

ライブ音声を録音するためのUIを立ち上げる前に、ダウンロードしたMP3をいくつか使って `transcribe()` 関数を実行してみましょう。クリアな音声の例として録音した `mary.mp3` からは `Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.` という認識結果が出力されます。2つ目のファイルは非常に歪んだ音声で、聞き取ることがかなり難しいですが、モデルは`Daisy, Daisy, give me your answer too. I'm half crazy all for the love of you.`と正しく認識しています。

注：このノートのデフォルトの長さである30秒でトランスクリプトが切れていることに気づくでしょう（設定の変更は可能）。

In [ ]:
easy_text = transcribe("/content/openai-whisper-webapp/mary.mp3")
print(easy_text)

hard_text = transcribe("/content/openai-whisper-webapp/daisy_HAL_9000.mp3")
print(hard_text)

## Web UIツールキットをインストールする

音声録音に必要なウィジェットを提供するためにgradioを使います。

In [ ]:
!pip install gradio==3.50

In [ ]:
import gradio as gr
import time

## ウェブインターフェース

このスクリプトを実行すると、冒頭で説明したように、ライブ音声の録音と書き起こしの確認に使用できる2つのウィジェットが下に表示されるはずです。

In [ ]:

gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI',
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()